# Module 1 Homework

In [4]:
!pip install yfinance

In [21]:
import pandas as pd
from datetime import datetime
from datetime import date

import yfinance as yf

### Question 1: [Index] S&P 500 Stocks Added to the Index


**Which year had the highest number of additions?**

Using the list of S&P 500 companies from Wikipedia's [S&P 500 companies page](https://en.wikipedia.org/wiki/List_of_S%26P_500_companies), download the data including the year each company was added to the index.

Hint: you can use [pandas.read_html](https://pandas.pydata.org/docs/reference/api/pandas.read_html.html) to scrape the data into a DataFrame.

Steps:
1. Create a DataFrame with company tickers, names, and the year they were added.
2. Extract the year from the addition date and calculate the number of stocks added each year.
3. Which year had the highest number of additions (1957 doesn't count, as it was the year when the S&P 500 index was founded)? Write down this year as your answer (the most recent one, if you have several records).

*Context*:
> "Following the announcement, all four new entrants saw their stock prices rise in extended trading on Friday" - recent examples of S&P 500 additions include DASH, WSM, EXE, TKO in 2025 ([Nasdaq article](https://www.nasdaq.com/articles/sp-500-reshuffle-dash-tko-expe-wsm-join-worth-buying)).

*Additional*: How many current S&P 500 stocks have been in the index for more than 20 years? When stocks are added to the S&P 500, they usually experience a price bump as investors and index funds buy shares following the announcement.

In [96]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
df = pd.read_html(url)[0]

subset_columns = ['Symbol', 'Security', 'Date added']
df = df.loc[:, subset_columns]

df['Year added'] = pd.to_datetime(df['Date added']).dt.year
counts = df.groupby('Year added').size().reset_index().sort_values(by=[0, 'Year added'], ascending=False).head()
counts


,Year added,0
0,1957,53
48,2017,23
47,2016,23
50,2019,22
39,2008,17


### Question 2. [Macro] Indexes YTD (as of 1 May 2025)

**How many indexes (out of 10) have better year-to-date returns than the US (S&P 500) as of May 1, 2025?**

Using Yahoo Finance World Indices data, compare the year-to-date (YTD) performance (1 January-1 May 2025) of major stock market indexes for the following countries:
* United States - S&P 500 (^GSPC)
* China - Shanghai Composite (000001.SS)
* Hong Kong - HANG SENG INDEX (^HSI)
* Australia - S&P/ASX 200 (^AXJO)
* India - Nifty 50 (^NSEI)
* Canada - S&P/TSX Composite (^GSPTSE)
* Germany - DAX (^GDAXI)
* United Kingdom - FTSE 100 (^FTSE)
* Japan - Nikkei 225 (^N225)
* Mexico - IPC Mexico (^MXX)
* Brazil - Ibovespa (^BVSP)

*Hint*: use start_date='2025-01-01' and end_date='2025-05-01' when downloading daily data in yfinance

Context:
> [Global Valuations: Who's Cheap, Who's Not?](https://simplywall.st/article/beyond-the-us-global-markets-after-yet-another-tariff-update) article suggests "Other regions may be growing faster than the US and you need to diversify."

Reference: Yahoo Finance World Indices - https://finance.yahoo.com/world-indices/

*Additional*: How many of these indexes have better returns than the S&P 500 over 3, 5, and 10 year periods? Do you see the same trend?
Note: For simplicity, ignore currency conversion effects.)


In [22]:
# United States - S&P 500 (^GSPC)
# China - Shanghai Composite (000001.SS)
# Hong Kong - HANG SENG INDEX (^HSI)
# Australia - S&P/ASX 200 (^AXJO)
# India - Nifty 50 (^NSEI)
# Canada - S&P/TSX Composite (^GSPTSE)
# Germany - DAX (^GDAXI)
# United Kingdom - FTSE 100 (^FTSE)
# Japan - Nikkei 225 (^N225)
# Mexico - IPC Mexico (^MXX)
# Brazil - Ibovespa (^BVSP)

tickers = [
    '^GSPC',
    '000001.SS',
    '^HSI',
    '^AXJO',
    '^NSEI',
    '^GSPTSE',
    '^GDAXI',
    '^FTSE',
    '^N225',
    '^MXX',
    '^BVSP'
]

start_dt = date(2025, 1,1)
end_dt = date(2025, 5, 1)

In [183]:
tickers_obj = yf.Tickers(tickers)

gt_GCSP_count = 0

for ticker in tickers:
  df = tickers_obj.tickers[ticker].history(start=start_dt, end=end_dt)
  df['Year'] = df.index.year
  df['YTD_Return'] = df.groupby('Year')['Close'].transform(lambda x: x/x.iloc[0] - 1)

  if ticker == '^GSPC':
    GSPC_YTD_Return = df.iloc[-1]['YTD_Return']

  ticker_YTD_Return = df.iloc[-1]['YTD_Return']
  print(f"{ticker:10} {(ticker_YTD_Return > GSPC_YTD_Return):^3} {ticker_YTD_Return:+} ")

  if (ticker_YTD_Return > GSPC_YTD_Return):
    gt_GCSP_count+=1

print(gt_GCSP_count)

^GSPC       0  -0.05103300748245043 
000001.SS   1  +0.005048172440500043 
^HSI        1  +0.1272001783592147 
^AXJO       1  -0.009145002952478443 
^NSEI       1  +0.02490423740978498 
^GSPTSE     1  -0.002261257179291487 
^GDAXI      1  +0.12346378381499523 
^FTSE       1  +0.028425901908435414 
^N225       0  -0.08297930804200304 
^MXX        1  +0.13049444457570325 
^BVSP       1  +0.12438709677419357 
9


### Question 3. [Index] S&P 500 Market Corrections Analysis



**Calculate the median duration (in days) of significant market corrections in the S&P 500 index.**

For this task, define a correction as an event when a stock index goes down by **more than 5%** from the closest all-time high maximum.

Steps:
1. Download S&P 500 historical data (1950-present) using yfinance
2. Identify all-time high points (where price exceeds all previous prices)
3. For each pair of consecutive all-time highs, find the minimum price in between
4. Calculate drawdown percentages: (high - low) / high × 100
5. Filter for corrections with at least 5% drawdown
6. Calculate the duration in days for each correction period
7. Determine the 25th, 50th (median), and 75th percentiles for correction durations

*Context:*
> * Investors often wonder about the typical length of market corrections when deciding "when to buy the dip" ([Reddit discussion](https://www.reddit.com/r/investing/comments/1jrqnte/when_are_you_buying_the_dip/?rdt=64135)).

> * [A Wealth of Common Sense - How Often Should You Expect a Stock Market Correction?](https://awealthofcommonsense.com/2022/01/how-often-should-you-expect-a-stock-market-correction/)

*Hint (use this data to compare with your results)*: Here is the list of top 10 largest corrections by drawdown:
* 2007-10-09 to 2009-03-09: 56.8% drawdown over 517 days
* 2000-03-24 to 2002-10-09: 49.1% drawdown over 929 days
* 1973-01-11 to 1974-10-03: 48.2% drawdown over 630 days
* 1968-11-29 to 1970-05-26: 36.1% drawdown over 543 days
* 2020-02-19 to 2020-03-23: 33.9% drawdown over 33 days
* 1987-08-25 to 1987-12-04: 33.5% drawdown over 101 days
* 1961-12-12 to 1962-06-26: 28.0% drawdown over 196 days
* 1980-11-28 to 1982-08-12: 27.1% drawdown over 622 days
* 2022-01-03 to 2022-10-12: 25.4% drawdown over 282 days
* 1966-02-09 to 1966-10-07: 22.2% drawdown over 240 days